
Universitario: Flores Yampara Felix Antonio

Carrera: Ingenieria de Sistemas

Docente: Ing. Pacheco Lora Carlos Walter

In [ ]:
import os
import numpy as np
import pandas as pd

from matplotlib import pyplot

from scipy import optimize

from scipy.io import loadmat

%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from datetime import datetime

# Importar libreria para dividir entrenamiento y prueba
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Cargar los datos desde el archivo CSV usando Pandas
data = pd.read_csv('/content/drive/MyDrive/SIS420 1.2024/Laboratorios/lab3/Firewall_Rule_Classification.csv', delimiter=',')


In [ ]:
print(data)

        Unnamed: 0  Source Port  Destination Port  NAT Source Port  \
0                0        33313               443            29395   
1                1        53871               443            15841   
2                2        56628               443            13354   
3                3        50270               445                0   
4                4        64113                53            33491   
...            ...          ...               ...              ...   
262123      262123        61522             64147                0   
262124      262124        54276             39004                0   
262125      262125        56601               445                0   
262126      262126        53007               445                0   
262127      262127        37837             37965                0   

        NAT Destination Port  Bytes  Bytes Sent  Bytes Received  Packets  \
0                        443   9675        2389            7286       43   
1      

In [ ]:
#conversion de la columna que esta en objeto a enteros
#variables categoricas
columnas_categoricas = data.select_dtypes(include=['object']).columns

In [ ]:
#Procesamiento de frases o palabras
for columna in columnas_categoricas:
  le = LabelEncoder()
  data[columna] = le.fit_transform(data[columna])

In [ ]:
print(data)

        Unnamed: 0  Source Port  Destination Port  NAT Source Port  \
0                0        33313               443            29395   
1                1        53871               443            15841   
2                2        56628               443            13354   
3                3        50270               445                0   
4                4        64113                53            33491   
...            ...          ...               ...              ...   
262123      262123        61522             64147                0   
262124      262124        54276             39004                0   
262125      262125        56601               445                0   
262126      262126        53007               445                0   
262127      262127        37837             37965                0   

        NAT Destination Port  Bytes  Bytes Sent  Bytes Received  Packets  \
0                        443   9675        2389            7286       43   
1      

In [ ]:
#Para la division del dataset en entrenamiento y prueba del 80% y 20%
#respectivamente se aplico la libreria train_test_split que con eso nos
#posibilita crear los nuevos datasets

# Division de dataset en entrenamiento y prueba
entrenamiento_data, prueba_data = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
print(entrenamiento_data)

        Unnamed: 0  Source Port  Destination Port  NAT Source Port  \
197833      197833        56453             37965                0   
104343      104343        49775                80            50741   
166274      166274        48861                53            40652   
227517      227517        62423               445                0   
31043        31043        54758             35993                0   
...            ...          ...               ...              ...   
259178      259178        65180                53            28162   
103694      103694        45550                80            42274   
131932      131932        59552               445                0   
146867      146867        56263                53            57350   
121958      121958        56281                53            38618   

        NAT Destination Port  Bytes  Bytes Sent  Bytes Received  Packets  \
197833                     0     62          62               0        1   
104343 

In [ ]:
print(prueba_data)

        Unnamed: 0  Source Port  Destination Port  NAT Source Port  \
18129        18129        41348              5555                0   
144750      144750         4772             37965                0   
57938        57938        33654               443            14790   
143764      143764        53306               443            56801   
18294        18294        58638              5900                0   
...            ...          ...               ...              ...   
81871        81871        55892               445                0   
121702      121702        27005             27017            42794   
89427        89427        29972             50584                0   
37453        37453         1972             57470                0   
74391        74391        55962                53            37019   

        NAT Destination Port  Bytes  Bytes Sent  Bytes Received  Packets  \
18129                      0     60          60               0        1   
144750 

In [ ]:
#Creacion de los nuevos datasets entrenamiento y prueba

entrenamiento_data.to_csv("entrenamiento.csv", index=False)

In [ ]:
prueba_data.to_csv("prueba.csv", index=False)

In [ ]:
#Carga del datasets del entrenamiento detectando que nuestra variable y que son
#nuestras etiquetas se encuentra en la segunda columna

data = np.loadtxt("/content/entrenamiento.csv", delimiter=',', skiprows=1)
#print(data)
#X, y = data[:,2:], data[:, 1]
#print(X.shape)
#print(y.shape)

In [ ]:
# En caso de identificar la "y" en la ultima posicion
#columnas_seleccionadas = list(range(0, 1)) + list(range(2, 786))
#columnas_seleccionadas = list(range(0, 785))
#X = data[:, columnas_seleccionadas]
X = data[:, :-1]
y = data[:, -1]
m = y.size

In [ ]:
print(X.shape)
print(y.shape)

(209702, 12)
(209702,)


In [ ]:
y[y == 10] = 0

In [ ]:
input_layer_size = 785
num_labels = 10
m = y.size

In [ ]:
# Selecciona aleatoriamente 100 puntos de datos para mostrar
rand_indices = np.random.choice(m, 100, replace=False)
sel = X[rand_indices, :]

#displayData(sel)

In [ ]:
def sigmoid(z):
    """
    Calcula la sigmoide de z.
    """
    return 1.0 / (1.0 + np.exp(-z))

In [ ]:
def lrCostFunction(theta, X, y, lambda_):
    """
    Calcula el costo de usar theta como parámetro para la regresión logística regularizada y
    el gradiente del costo w.r.t. a los parámetros.

    Parametros
    ----------
    theta : array_like
        Parametro theta de la regresion logistica. Vector de la forma(shape) (n, ). n es el numero de caracteristicas
        incluida la intercepcion

    X : array_like
        Dataset con la forma(shape) (m x n). m es el numero de ejemplos, y n es el numero de
        caracteristicas (incluida la intercepcion).

    y : array_like
        El conjunto de etiquetas. Un vector con la forma (shape) (m, ). m es el numero de ejemplos

    lambda_ : float
        Parametro de regularización.

    Devuelve
    -------
    J : float
        El valor calculado para la funcion de costo regularizada.

    grad : array_like
        Un vector de la forma (shape) (n, ) que es el gradiente de la
        función de costo con respecto a theta, en los valores actuales de theta..
    """
#     alpha = 0.003
#     theta = theta.copy()
    # Inicializa algunos valores utiles
    m = y.size

    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)

    J = 0
    grad = np.zeros(theta.shape)

    h = sigmoid(X.dot(theta.T))

    temp = theta
    temp[0] = 0

#     J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h)))
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    grad = (1 / m) * (h - y).dot(X)
#     theta = theta - (alpha / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp

    return J, grad
#    return J, theta

In [ ]:
def oneVsAll(X, y, num_labels, lambda_):
    """
    Trains num_labels logistic regression classifiers and returns
    each of these classifiers in a matrix all_theta, where the i-th
    row of all_theta corresponds to the classifier for label i.

    Parameters
    ----------
    X : array_like
        The input dataset of shape (m x n). m is the number of
        data points, and n is the number of features. Note that we
        do not assume that the intercept term (or bias) is in X, however
        we provide the code below to add the bias term to X.

    y : array_like
        The data labels. A vector of shape (m, ).

    num_labels : int
        Number of possible labels.

    lambda_ : float
        The logistic regularization parameter.

    Returns
    -------
    all_theta : array_like
        The trained parameters for logistic regression for each class.
        This is a matrix of shape (K x n+1) where K is number of classes
        (ie. `numlabels`) and n is number of features without the bias.
    """
    # algunas variables utiles
    m, n = X.shape

    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 50}
        res = optimize.minimize(lrCostFunction,
                                initial_theta,
                                (X, (y == c), lambda_),
                                jac=True,
                                method='CG',
                                options=options)

        all_theta[c] = res.x

    return all_theta

In [ ]:
#Carga de nuestro datasets prueba para iniciar nuestras predicciones
data = np.loadtxt("/content/prueba.csv", delimiter=',', skiprows=1)

In [ ]:
X = data[:, :-1]
y = data[:, -1]
m = y.size

In [ ]:
print(X.shape)
print(y.shape)

(52426, 12)
(52426,)


In [ ]:
lambda_ = 0.001
all_theta = oneVsAll(X, y, num_labels, lambda_)
print(all_theta.shape)

<ipython-input-113-575a438e2fc3>:5: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))
<ipython-input-114-cf6e0809223e>:49: RuntimeWarning: divide by zero encountered in log
  J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
<ipython-input-113-575a438e2fc3>:5: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))
<ipython-input-114-cf6e0809223e>:49: RuntimeWarning: divide by zero encountered in log
  J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
<ipython-input-113-575a438e2fc3>:5: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))
<ipython-input-114-cf6e0809223e>:49: RuntimeWarning: divide by zero encountered in log
  J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
<ipython-input-113-575a438e2fc3>:5: Runt

(10, 13)


<ipython-input-113-575a438e2fc3>:5: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))
<ipython-input-114-cf6e0809223e>:49: RuntimeWarning: divide by zero encountered in log
  J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))


In [ ]:
def predictOneVsAll(all_theta, X):
    """
    Devuelve un vector de predicciones para cada ejemplo en la matriz X.
    Tenga en cuenta que X contiene los ejemplos en filas.
    all_theta es una matriz donde la i-ésima fila es un vector theta de regresión logística entrenada para la i-ésima clase.
    Debe establecer p en un vector de valores de 0..K-1 (por ejemplo, p = [0, 2, 0, 1]
    predice clases 0, 2, 0, 1 para 4 ejemplos).

    Parametros
    ----------
    all_theta : array_like
        The trained parameters for logistic regression for each class.
        This is a matrix of shape (K x n+1) where K is number of classes
        and n is number of features without the bias.

    X : array_like
        Data points to predict their labels. This is a matrix of shape
        (m x n) where m is number of data points to predict, and n is number
        of features without the bias term. Note we add the bias term for X in
        this function.

    Devuelve
    -------
    p : array_like
        The predictions for each data point in X. This is a vector of shape (m, ).
    """

    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

    return p

In [ ]:
print(X.shape)
pred = predictOneVsAll(all_theta, X)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y) * 100))
XPrueba = X[10:150, :].copy()
print(XPrueba.shape)
#print(np.ones((1)))
#print(XPrueba)
#p = np.zeros(1)
XPrueba = np.concatenate([np.ones((140, 1)), XPrueba], axis=1)
print(XPrueba.shape)
p = np.argmax(sigmoid(XPrueba.dot(all_theta.T)), axis = 1)
print(p)

#displayData(X[1002:1003, :])
print(y[10:150])

(52426, 12)
Precision del conjuto de entrenamiento: 57.23%
(140, 12)
(140, 13)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0. 0. 2. 0. 2. 2. 0. 0. 0. 0. 0. 1. 0. 2. 0. 0. 0. 0. 2. 2. 1. 0. 0. 0.
 2. 0. 1. 1. 2. 1. 2. 0. 0. 0. 2. 0. 2. 1. 0. 0. 2. 0. 1. 0. 0. 0. 0. 0.
 0. 2. 0. 1. 0. 0. 2. 2. 0. 0. 0. 0. 1. 0. 2. 1. 0. 0. 0. 1. 0. 0. 2. 0.
 0. 2. 0. 0. 1. 1. 1. 1. 0. 1. 0. 0. 2. 0. 0. 0. 0. 0. 0. 2. 1. 0. 0. 0.
 2. 0. 2. 0. 1. 1. 2. 0. 0. 0. 0. 0. 2. 0. 0. 2. 0. 1. 1. 0. 0. 2. 0. 0.
 1. 1. 1. 2. 2. 2. 2. 0. 2. 1. 2. 0. 0. 2. 2. 0. 0. 2. 1. 1.]


<ipython-input-113-575a438e2fc3>:5: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))


Para la precision del modelo pudimos obtener un 57.23 % por ciento que es aceptable y demuestra que nuestras predicciones son correctas